# !nvidia-smi is used to monitor and manage the resources and status of NVIDIA GPUs. When run, it provides detailed information about GPU utilization, including driver version, memory usage, temperature, workload and running processes.

In [ ]:

!nvidia-smi

In [ ]:
import os

# Use your Hugging Face key here!
os.environ["HF_TOKEN"] = "YOUR_HF_TOKEN"

# Installation of the necessary dependencies.

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0 # Installs bitsandbytes for optimizing low-level operations with tensors, focused on memory efficiency and speed.
!pip3 install -q -U peft==0.8.2 # Installs peft, a tool for analyzing and optimizing the performance of Python code, especially for scientific computing.
!pip3 install -q -U trl==0.7.10 # Installs trl, a library for Reinforcement Learning in transformers, used for training language models.
!pip3 install -q -U accelerate==0.27.1 # Installs accelerate, a library for accelerating computing in PyTorch, simplifying execution on CPU or GPU.
!pip3 install -q -U datasets==2.17.0 # Installs datasets, a library for loading and manipulating datasets easily and efficiently, aimed at Machine Learning.
!pip3 install -q -U transformers==4.38.0 # Installs transformers, a library for pre-trained transformer models, focused on NLP (Natural Language Processing).

In [ ]:
# Making all imports here.
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer


# The code initializes and configures a Causal Language Model (CLM) using the Transformers library with a specific 4-bit quantization configuration provided by the BitsAndBytes library. The selected model is "google/gemma-7b," a pre-trained transformer model.

**model_id:** variable stores the identifier of the pre-trained model to be loaded, in this case, "google/gemma-7b."

**bnb_config:** is a configuration for bit quantization provided by the BitsAndBytes library. This setup enables the model to be loaded in 4-bit precision, using a specific quantization type (nf4), computing data in float16, and enabling double quantization for better accuracy.

**model:** is the variable that holds the loaded model with the defined quantization settings and the device mapping indicating that the model will be loaded on GPU 0 (if available).

**tokenizer:** is responsible for converting text into tokens that the model can understand, using the same model identifier to ensure compatibility. The option add_eos_token=True signifies that an end-of-sequence token should be added to tokenized texts.

The aim is to prepare a transformer model for text generation with an efficient memory configuration (through quantization) and ready for natural language tasks, such as text generation, ensuring compatibility between the model and the tokenizer for proper processing of input and output text.

In [ ]:
model_id = "google/gemma-7b"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=getattr(torch, "float16"),
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

# The code snippet is designed to prepare a dataset for a text-to-SQL query translation task, where the model is trained to generate SQL queries based on English questions within a given context. The process involves creating prompts from the dataset and then splitting it for training and testing.

**generate_prompt function:** This function builds a formatted message for each data point in the dataset. It starts with a prefix text that explains the task (translation from text to SQL) and then includes the context and the user's question, constructs a message that includes the context, the question and the answer; otherwise, it constructs a message, but treats it in the same way due to the fact that the if-else condition is effectively identical.

**Loading the dataset:** The dataset is loaded using load_dataset from a specific source ("b-mc2/sql-create-context") and split (in this case, using the "train" split). Here is the link to the dataset on Hugging Face: https://huggingface.co/datasets/b-mc2/sql-create-context/viewer/default/train?p=1

**Generating text prompts:** For each data point in the dataset, the generate_prompt function is called to create a formatted text prompt. These prompts are then stored in text_column.

**Adding a new column:** The dataset is augmented by adding a new column named "prompt", which contains the generated text prompts.

**Shuffling the dataset:** The dataset is shuffled to ensure a random distribution of data points, which is beneficial for training models to prevent the model from learning any unintentional patterns from the order of the data.

**Tokenizing:** The dataset is then processed using a tokenizer (presumably defined earlier but not shown in the snippet), which converts the text prompts into a format suitable for model training. This is done in batches for efficiency.

**Splitting the dataset:** Finally, the dataset is split into training and testing sets, with 20% of the data being reserved for testing. This split allows for the evaluation of the model on unseen data, ensuring that the model can generalize well beyond the training dataset.

Overall, this code prepares a dataset for training a model to translate English questions into SQL queries, with a focus on context-aware translation. The preparation involves formatting, shuffling, tokenizing, and splitting the dataset to facilitate effective model training and evaluation.

In [ ]:
def generate_prompt(data_point):
    return f"""<start_of_turn>You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided CONTEXT: {data_point["context"]}.
        
    user {data_point["question"]} <end_of_turn><start_of_turn>model{data_point["answer"]} <end_of_turn>
    """

# add the "prompt" column in the dataset
dataset = load_dataset("b-mc2/sql-create-context", split="train")
text_column = [generate_prompt(data_point) for data_point in dataset]

dataset = dataset.add_column("prompt", text_column)
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset.train_test_split(test_size=0.2)


# The code snippet below is related to the enhancement of a pre-existing model using LoRA (Low-Rank Adaptation), facilitated by the peft (Python Efficient Fine-Tuning) library. LoRA is a technique for fine-tuning large language models in a parameter-efficient manner, by introducing trainable low-rank matrices to model weights.

1) Import Statements: The snippet begins by importing LoraConfig and get_peft_model from the peft library. LoraConfig is used to define the configuration for the LoRA adaptation, while get_peft_model is a function that applies this configuration to a specified model.

2) LoRA Configuration: A LoraConfig object is instantiated with several parameters:

- r: Rank of the low-rank matrices. This defines the size of the bottleneck in the LoRA adaptation, impacting the model's capacity and efficiency.

- lora_alpha: A scaling factor applied to the LoRA parameters, controlling the magnitude of the updates.

- target_modules: A list of module names within the model where LoRA will be applied. These typically correspond to specific parts of the transformer architecture, like projection layers (o_proj, q_proj, etc.).

-lora_dropout: Dropout rate applied to the LoRA parameters, which can help in preventing overfitting.

- bias: Specifies how biases are treated in the adaptation process. In this case, it is set to "none", indicating no bias adjustments.

- task_type: The type of task the model is intended for, with "CAUSAL_LM" indicating a causal language modeling task, where the model generates text based on previous tokens.

3) Model Adaptation: The get_peft_model function is called with the original model and the LoRA configuration. This function adapts the specified model by applying the LoRA parameters according to the lora_config. The result is a model enhanced with LoRA, ready for efficient fine-tuning on specific tasks with potentially lower resource requirements compared to full model fine-tuning.

Overall, this code is used to apply LoRA to a pre-existing model for more efficient fine-tuning, targeting specific parts of the model as defined in the lora_config, with the aim of enhancing its performance on a causal language modeling task.

In [ ]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    bias="none",
    tas
    k_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

# The code snippet below demonstrates how to fine-tune a pre-existing model using the SFTTrainer, which is likely a custom or extended trainer from the transformers library, tailored for efficient fine-tuning methods like LoRA (Low-Rank Adaptation). The process involves setting up the trainer with specific configurations and then starting the training process. Finally, the fine-tuned model is saved.

1) **Tokenizer Configuration:** The tokenizer's padding token is set to the same value as its end-of-sequence (EOS) token. This ensures consistent tokenization behavior, especially for models that rely on specific token types for understanding text structure.

2) **Clearing CUDA Cache:** torch.cuda.empty_cache() is called to free unused memory from the GPU, potentially improving memory utilization for the upcoming training session.

3) **SFTTrainer Setup:**

- **model** and **lora_config** are passed as arguments, indicating the model to be fine-tuned and the LoRA configuration for efficient parameter updates.

- The training and evaluation datasets are specified from a previously loaded or prepared dataset, segmented into **"train"** and **"test"** splits.

- **dataset_text_field="prompt"** indicates that the text field named "prompt" in the dataset should be used for training.

- Training arguments are configured with parameters such as batch size, learning rate, optimizer type (**"paged_adamw_8bit"** for memory-efficient optimization), and the output directory for saving training logs and model checkpoints.

- A **data_collator** is provided for batching and preparing data during training. In this case, it's configured for language modeling without masked language modeling (MLM=false).

4) **Training: The train()** method is called on the trainer to start the fine-tuning process. This will train the model according to the configurations specified, using the training dataset and evaluating its performance on the test dataset.

5) **Saving the Fine-tuned Model:** After training, the fine-tuned model is saved with a new name ("gemma-7b-sqlexpert"). This step makes the model available for future use, including uploading to the Hugging Face Model Hub for easy access and sharing.

This code illustrates an advanced fine-tuning approach leveraging specific training strategies and configurations for efficient training, aimed at enhancing a model's performance on a targeted task. The use of a specialized trainer and fine-tuning techniques like LoRA suggests an emphasis on maintaining or improving model performance while managing computational and memory resources effectively.

In [ ]:
#new code using SFTTrainer
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# Start the training process
trainer.train()

new_model = "gemma-7b-sqlexpert" #Name of the model you will be pushing to huggingface model hub
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)


# The code snippet bellow demonstrates the process of merging a base model with modifications (likely including LoRA weights) into a single consolidated model, followed by saving the merged model and configuring the associated tokenizer. This process is part of fine-tuning and optimizing models for specific tasks or improving their efficiency. Let's break down the steps:

1) **Loading the Base Model:** The AutoModelForCausalLM.from_pretrained method is used to load a pre-existing base model identified by model_id. Several parameters are specified for efficient loading and operation:

- low_cpu_mem_usage=True: Reduces CPU memory usage by loading model weights on demand rather than all at once. This is beneficial for systems with limited memory.

- return_dict=True: Ensures that the model outputs are returned as a dictionary, providing easier access to different components of the output.

- torch_dtype=torch.float16: Sets the data type of the model tensors to float16, reducing memory usage and potentially increasing computation speed on compatible hardware.

- device_map={"": 0}: Maps the model to a specific device, in this case, GPU 0. This ensures that the model utilizes GPU acceleration if available.

2) **Merging the Model with LoRA Weights:** PeftModel.from_pretrained is called with the base model and the path or identifier of the new model (new_model) containing LoRA or other parameter-efficient fine-tuning (PEFT) weights. This step creates an instance of PeftModel that includes both the base model and the modifications.

- merged_model.merge_and_unload(): This method finalizes the merging process of the base model with the additional weights or modifications and prepares the merged_model for use or saving by unloading any temporary resources.

3) **Saving the Merged Model:** The save_pretrained method saves the merged_model to a directory named "merged_model". The safe_serialization=True parameter ensures that the model is saved in a way that minimizes risks during the saving process, potentially by using more robust serialization methods.

4) **Configuring and Saving the Tokenizer:** The tokenizer associated with the model is also saved in the "merged_model" directory. Before saving, the tokenizer's padding token is set to be the same as its end-of-sequence (EOS) token, and the padding is configured to be applied from the "right" side. These tokenizer settings are crucial for ensuring consistent text processing, especially for models trained on specific tokenization and padding schemes.

This process is crucial for deploying models that have been fine-tuned with advanced techniques like LoRA, ensuring that all modifications are properly integrated with the base model and that the model is ready for inference or further training. Saving the model and tokenizer in this way facilitates easy sharing and reuse.

In [ ]:
# Merge the model with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# The code below defines and utilizes a function, get_completion, to generate SQL queries from natural language descriptions using a pre-trained model and tokenizer. The function is structured to convert English queries into corresponding SQL commands by providing contextual information about a database schema.

1) **Function Definition:** get_completion takes four parameters: context (a string describing the database schema), query (a natural language question or statement), model (the pre-trained model capable of generating SQL queries), and tokenizer (used to convert strings to model-understandable tokens and vice versa).

2) **Device Setting:** It sets the device to cuda:0 to utilize the first GPU for model inference, ensuring faster processing.

3) **Prompt Template:** A template string is defined to structure the input to the model. It introduces the task to the model ("You are a text to SQL query translator") and formats the provided context and query into a coherent prompt for the model.

4) **Tokenization:** The tokenizer converts the prompt into tokens with return_tensors="pt" to get PyTorch tensors, and add_special_tokens=True to include tokens necessary for the model's understanding (like start-of-sequence).

5) **Model Inference:** The tokenized input is moved to the GPU (encodeds.to(device)) and passed to the model. The generate method is called with parameters controlling the generation process: max_new_tokens specifies the maximum length of the generated sequence, do_sample=True enables stochastic generation for varied outputs, and pad_token_id is set to ensure proper sequence termination.

6) **Decoding:** The generated token IDs are decoded back into a string (decoded), excluding special tokens for clarity.

7) **Usage:** The function is then called with a specific context detailing the schema of a bookstore's database and a query that describes the database setup in natural language. The model and tokenizer arguments are assumed to be previously initialized and passed to the function.

7 **Output:** Finally, the function returns the generated SQL query as a string, and the result is printed out.

In essence, this code leverages a machine learning model's capability to understand complex natural language instructions and generate corresponding SQL queries, demonstrating a practical application of language models in database management and querying.

In [ ]:
def get_completion(context:str, query: str, model, tokenizer) -> str:
  device = "cuda:0"
  prompt_template = """
  <start_of_turn>user
  You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided CONTEXT: {context}

  
  {query}
  <end_of_turn>
  <start_of_turn>model
  
  """
  prompt = prompt_template.format(context=context, query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

result = get_completion(
    context="""
CREATE TABLE Books (
    BookID INT PRIMARY KEY,
    Title VARCHAR(100),
    Author VARCHAR(100),
    Price DECIMAL(10, 2)
);

CREATE TABLE Sales (
    SaleID INT PRIMARY KEY,
    BookID INT,
    SaleDate DATE,
    Quantity INT,
    FOREIGN KEY (BookID) REFERENCES Books(BookID)
);
""",
    query="""Imagine you have a database for a bookstore that includes two tables. The first table, Books, contains details about the books available in the store, including BookID, Title, Author, and Price. The second table, Sales, records each sale, including SaleID, BookID, SaleDate, and Quantity.""", 
    model=merged_model, 
    tokenizer=tokenizer
  )
print(result)